In [2]:
import numpy as np
import os
from keras.layers import Conv3D, MaxPooling3D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Activation, ZeroPadding3D, TimeDistributed, LSTM, GRU, Reshape
#from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.preprocessing import LabelEncoder
import sklearn
import seaborn as sns
import tensorflow as tf
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from prettytable import PrettyTable
from matplotlib import pyplot as plt

import time

# Vgg Model Building


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16

# Define constants
IMAGE_SIZE = (96, 72)
BATCH_SIZE = 16
EPOCHS = 100
NUM_CLASSES = 17
TRAIN_DATA_DIR = "D:\\New Dataset Lip Movement Projec\\FINAL YEAR PROJECT\\output_denoised_image_folder"

# Define data generator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Load and preprocess the data with splitting
train_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  # 80% of the data will be used for training
)

validation_generator = datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  # 20% of the data will be used for validation
)

# Load pre-trained VGG-16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Define the CNN model with VGG-16 base
model = Sequential([
    base_model,
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Found 246 images belonging to 17 classes.
Found 52 images belonging to 17 classes.



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 2, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 4096)              12587008  
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 17)                69649     
                                                                 
Total params: 44152657 (168.43 MB)
Trainable params: 29437969 (112.30 MB)
Non-trainable params: 147146

In [5]:
start_time=time.time()
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)
# Stop the timer
end_time = time.time()

# Compute the computational time
comp_time = end_time - start_time

model.save('denoised_vgg16_model11.h5')

# Evaluate the model
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(validation_generator)

print(f'Training Accuracy: {train_acc * 100}')
print(f'Validation Accuracy: {val_acc * 100}')
print(f'Training time: {comp_time}')


Epoch 1/100
15/15 [==============================] - 10s 667ms/step - loss: 3.6493 - accuracy: 0.0913 - val_loss: 2.6903 - val_accuracy: 0.1458
Epoch 2/100
15/15 [==============================] - 11s 688ms/step - loss: 2.5532 - accuracy: 0.1739 - val_loss: 2.3608 - val_accuracy: 0.2083
Epoch 3/100
15/15 [==============================] - 10s 676ms/step - loss: 2.0410 - accuracy: 0.3522 - val_loss: 2.0244 - val_accuracy: 0.3542
Epoch 4/100
15/15 [==============================] - 10s 646ms/step - loss: 1.6472 - accuracy: 0.4565 - val_loss: 1.9587 - val_accuracy: 0.3125
Epoch 5/100
15/15 [==============================] - 10s 667ms/step - loss: 1.3122 - accuracy: 0.5609 - val_loss: 1.5593 - val_accuracy: 0.4167
Epoch 6/100
15/15 [==============================] - 9s 633ms/step - loss: 1.0559 - accuracy: 0.6565 - val_loss: 1.3657 - val_accuracy: 0.4792
Epoch 7/100
15/15 [==============================] - 9s 628ms/step - loss: 0.8486 - accuracy: 0.6913 - val_loss: 1.2587 - val_accuracy: 0

c:\Users\ksaqu\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 1s 216ms/step - loss: 6.8586e-04 - accuracy: 1.0000
Training Accuracy: 100.0
Validation Accuracy: 100.0
Training time: 953.2928850650787
